In [ ]:
#安裝SBTi模組步驟1
%pip install sbti-finance-tool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.4
    Uninstalling pydantic-2.6.4:
      Successfully uninstalled pydantic-2.6.4


In [ ]:
#安裝SBTi模組步驟2
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd
%matplotlib inline

import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

In [ ]:
#匯入資料步驟12
from google.colab import files
uploaded = files.upload()

In [ ]:
#匯入資料步驟12
from google.colab import files
uploaded = files.upload()

In [ ]:
#匯入資料步驟13 請注意這邊要放Provider
provider = ExcelProvider('Provider-test.xlsx')

In [ ]:
#匯入資料步驟13 請注意這邊要放Portfolio
df_portfolio = pd.read_excel('Portfolio-test.xlsx')

In [ ]:
#匯入資料 預覽匯入之Portfolio
df_portfolio.head(5)

In [ ]:
#計算溫度數值步驟1
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)
temperature_score = TemperatureScore(
    time_frames=list(SBTi.interfaces.ETimeFrames),
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3], # EScopes: S3, S1S2 and S1S2S3
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

In [ ]:
#計算溫度數值步驟2
details = amended_portfolio[['company_name', 'time_frame', 'scope', 'temperature_score']]
print(details)

In [ ]:
#計算溫度數值步驟3
#無須下載可以先跳過
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#計算溫度數值步驟4
#無須下載可以先跳過
details.to_csv('/content/drive/MyDrive/details.csv')

In [ ]:
#計算溫度數值步驟6
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))

In [ ]:
#加權方式比較步驟1
portfolio_data = SBTi.utils.get_data([provider], companies)
scenarios = {}
scores_collection = {}
temperature_score = TemperatureScore(time_frames=list(SBTi.interfaces.ETimeFrames), scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3])
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

In [ ]:
#加權方式比較步驟2
#WATS
temperature_score.aggregation_method = PortfolioAggregationMethod.WATS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_wats = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'WATS': df_wats})
#TETS
temperature_score.aggregation_method = PortfolioAggregationMethod.TETS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_tets = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'TETS': df_tets})
#MOTS
temperature_score.aggregation_method = PortfolioAggregationMethod.MOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_mots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'MOTS': df_mots})
#ETOS
temperature_score.aggregation_method = PortfolioAggregationMethod.EOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_eots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'EOTS': df_eots})
#ECOTS
temperature_score.aggregation_method = PortfolioAggregationMethod.ECOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_ecots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'ECOTS': df_ecots})
#AOTS
temperature_score.aggregation_method = PortfolioAggregationMethod.AOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_aots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'AOTS': df_aots})
#ROTS
temperature_score.aggregation_method = PortfolioAggregationMethod.ROTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_rots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'ROTS': df_rots})
pd.concat(scores_collection, axis=0)

In [ ]:
#情境分析步驟1-短期
# Set the time frames and scopes for the scoring
time_frames = [SBTi.interfaces.ETimeFrames.SHORT]         # ETimeFrames: SHORT MID and LONG
scopes = [EScope.S1S2]                                # EScopes: S1, S2, S3, S1S2 and S1S2S3
#Base scenario(temperature scores for all the timeframe and scope combinations)
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes)
base_scenario = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
base_score = base_scenario_aggregated.short.S1S2.all.score
scenarios.update({'Base Scenario-short': base_score})
print("The portfolio's base scenario-short temperature score (short - S1S2) is {c:.2f} degrees celcius".format(c=base_score))

In [ ]:
#情境分析步驟2-短期
#Scenario 2-short(engage with companies to influence them to set SBTi approved emissions reduction targets)
#maximum score=1.75
scenario = Scenario()
scenario.scenario_type = ScenarioType.APPROVED_TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_2_short_score = scenario_aggregated.short.S1S2.all.score
scenarios.update({'Scenario 2-short': scenario_2_short_score})
print("The portfolio's scenario 2-short temperature score (short - S1S2) is {c:.2f} degrees Celsius".format(c=scenario_2_short_score))

In [ ]:
#情境分析步驟3-短期
#Scenario 4b-short(engage with any company in the portfolio and manage to influence them to set SBTi-approved targets)
#maximum score=1.75
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_4b1_score = scenario_aggregated.short.S1S2.all.score
scenarios.update({'Scenario 4b-short': scenario_4b1_score})
print("The portfolio's scenario 4b-short temperature score (short - S1S2) is {c:.2f} degrees Celsius".format(c=scenario_4b1_score))


In [ ]:
#情境分析步驟1-中期
# Set the time frames and scopes for the scoring
time_frames = [SBTi.interfaces.ETimeFrames.MID]         # ETimeFrames: SHORT MID and LONG
scopes = [EScope.S1S2]                                # EScopes: S1, S2, S3, S1S2 and S1S2S3
#Base scenario(temperature scores for all the timeframe and scope combinations)
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes)
base_scenario = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
base_score = base_scenario_aggregated.mid.S1S2.all.score
scenarios.update({'Base Scenario-mid': base_score})
print("The portfolio's base scenario-mid temperature score (mid - S1S2) is {c:.2f} degrees celcius".format(c=base_score))

In [ ]:
#情境分析步驟2-中期
#Scenario 2-mid(engage with companies to influence them to set SBTi approved emissions reduction targets)
#maximum score=1.75
scenario = Scenario()
scenario.scenario_type = ScenarioType.APPROVED_TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_2_mid_score = scenario_aggregated.mid.S1S2.all.score
scenarios.update({'Scenario 2-mid': scenario_2_mid_score})
print("The portfolio's scenario 2-mid temperature score (mid - S1S2) is {c:.2f} degrees Celsius".format(c=scenario_2_mid_score))

In [ ]:
#情境分析步驟3-中期
#Scenario 4b-mid(engage with any company in the portfolio and manage to influence them to set SBTi-approved targets)
#maximum score=1.75
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_4b2_score = scenario_aggregated.mid.S1S2.all.score
scenarios.update({'Scenario 4b-mid': scenario_4b2_score})
print("The portfolio's scenario 4b-mid temperature score (mid - S1S2) is {c:.2f} degrees Celsius".format(c=scenario_4b2_score))

In [ ]:
#情境分析步驟1-長期
# Set the time frames and scopes for the scoring
time_frames = [SBTi.interfaces.ETimeFrames.LONG]         # ETimeFrames: SHORT MID and LONG
scopes = [EScope.S1S2]                                # EScopes: S1, S2, S3, S1S2 and S1S2S3
#Base scenario(temperature scores for all the timeframe and scope combinations)
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes)
base_scenario = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
base_score = base_scenario_aggregated.long.S1S2.all.score
scenarios.update({'Base Scenario-long': base_score})
print("The portfolio's base scenario-long temperature score (long - S1S2) is {c:.2f} degrees celcius".format(c=base_score))

In [ ]:
#情境分析步驟2-長期
#Scenario 2-L(engage with companies to influence them to set SBTi approved emissions reduction targets)
#maximum score=1.75
scenario = Scenario()
scenario.scenario_type = ScenarioType.APPROVED_TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_2_long_score = scenario_aggregated.long.S1S2.all.score
scenarios.update({'Scenario 2-long': scenario_2_long_score})
print("The portfolio's scenario 2-long temperature score (long - S1S2) is {c:.2f} degrees Celsius".format(c=scenario_2_long_score))

In [ ]:
#情境分析步驟3-長期
#Scenario 4b-long(engage with any company in the portfolio and manage to influence them to set SBTi-approved targets)
#maximum score=1.75
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_4b3_score = scenario_aggregated.long.S1S2.all.score
scenarios.update({'Scenario 4b-long': scenario_4b3_score})
print("The portfolio's scenario 4b-long temperature score (long - S1S2) is {c:.2f} degrees Celsius".format(c=scenario_4b3_score))

In [ ]:
#情境分析步驟4
print('{:<20s}{}'.format('', 'Temp score'))
print('{:<20s}{}'.format('Scenario overview', '(S1S2):'))
for scenario, score in scenarios.items():
    print('{:<21s}{:.2f}'.format(scenario, score))